In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### ITEM 20: PREFER RAISING EXCEPTIONS TO RETURNING NONE

In [9]:
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

In [11]:
def careful_divide(a, b):
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')

In [12]:
x, y = 5, 0
try:
    result = careful_divide(x, y)
except ValueError:
    print('Invalid inputs')
else:
    print('Result is %.1f' % result)

Invalid inputs


### ITEM 21: KNOW HOW CLOSURES INTERACT WITH VARIABLE SCOPE

In [15]:
def sort_priority(values, group):
    def helper(x):
        print(x)
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

In [19]:
numbers.sort??

In [16]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

8
3
1
2
5
4
7
6
[2, 3, 5, 7, 1, 4, 6, 8]


### this is easier to understand

In [27]:
class Sorter:
    def __init__(self, group):
        self.group = group
        self.found = False

    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)

sorter = Sorter(group)
print(sorter)
numbers.sort(key=sorter)
assert sorter.found is True
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


When you reference a variable in an expression, the Python interpreter traverses the scope to resolve the reference in this order:

The current function’s scope.

Any enclosing scopes (such as other containing functions).

The scope of the module that contains the code (also called the global scope).

The built-in scope (that contains functions like len and str).

In [31]:
not []
not None
not 0
not False

True

True

True

True

In [33]:
def print_parameters(**kwargs):
    for key, value in kwargs.items():
        print(f'{key} = {value}')

print_parameters(alpha=1.5, beta=9, gamma=4)
print_parameters(alpha=1.5, beta=9, gamma=4, sigma=66)

alpha = 1.5
beta = 9
gamma = 4
alpha = 1.5
beta = 9
gamma = 4
sigma = 66


In [39]:
def trace(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print(f'{func.__name__}({args!r}, {kwargs!r}) '
              f'-> {result!r}')
        return result
    return wrapper

In [41]:
@trace # equivalent to fibonacci = trace(fibonacci)
def fibonacci(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))

In [42]:
fibonacci(4)

fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((3,), {}) -> 2
fibonacci((4,), {}) -> 3


3

In [43]:
trace(fibonacci(4))

fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 1
fibonacci((3,), {}) -> 2
fibonacci((4,), {}) -> 3


<function __main__.trace.<locals>.wrapper(*args, **kwargs)>

### position and keyword arguments

In [54]:
# *args, unknown number of position argumeents
# **kargs, unknown number of keyword arguments
def some_func(*args, **kargs):
    print(args[0]+args[1], args, kargs)
    
some_func(1, 2, 'arg1', kargs1='arg2')

3 (1, 2, 'arg1') {'kargs1': 'arg2'}


In [60]:
def some_func(a, b, k1=10, k2=100):
    print(a+b+k1+k2)
    
some_func(1, 2)
some_func(1, 2, 3)
some_func(1, 2, k2=3)

113
106
16


In [4]:
!python --version

Python 3.8.1


In [28]:
# arguments before / are position-only
# arguments after * are keyword-only
# in between can be either
def some_func(a, b, /, c, *, k1=10, k2=100):
    print(a+b+k1+k2)
    

some_func(1, 2, 3)
some_func(1, 2, c=2, k2=3)
some_func(1, 2, 2, k2=3)

113
16
16


### list, set, dictionary comprehension and generator expression

In [30]:
stock = {
    'nails': 125,
    'screws': 35,
    'wingnuts': 8,
    'washers': 24,
}

order = ['screws', 'wingnuts', 'clips']

def get_batches(count, size):
    return count // size

In [36]:
stock.get??

In [35]:
# get value by key, default to 0
{name:stock.get(name, 0) for name in order}

{'screws': 35, 'wingnuts': 8, 'clips': 0}

In [46]:
# walrus assignment :=
found = {name: batches for name in order
         if (batches := get_batches(stock.get(name, 0), 8))}

found

{'screws': 4, 'wingnuts': 1}

In [45]:
# why it does not output clips: 0
{name: get_batches(stock.get(name, 0), 8) for name in order}

{'screws': 4, 'wingnuts': 1, 'clips': 0}

### consider generators instead of returning list

In [61]:
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
             yield index + 1

In [66]:
# yield produces generator, it returns a item at a time when you call next build-in method
address = 'Four score and seven years ago...'
it = index_words_iter(address)
print(next(it))
print(next(it))

list(it)

0
5


[11, 15, 21, 27]

In [67]:
list(enumerate(address))[:2]

[(0, 'F'), (1, 'o')]

In [92]:
# generator is exhausted after you read each element once, can not be reused
# does not consume huge amount of memories, one a line in the memory
# good for data streaming
# generator can be composed together

nums = list(range(100))
it = (len(x) for x in open('wages.txt'))
# next(it)
# next(it)
# next(it)

double = ((x, x*2) for x in it)
# next(double)
# next(double)
# next(double)

In [95]:
def yf(it):
    yield from it
    
list(yf(it))

[]

In [98]:
import itertools
# help(itertools)